In [166]:
import pickle
import numpy as np
import pandas as pd
from geographiclib.geodesic import Geodesic

In [167]:
import random
 
def generate_random_array(length):
    array = []
    for _ in range(length):
        array.append(random.randint(0, 1))
    return array
 
# 生成长度为10的随机数组
random_array = generate_random_array(10)
print(random_array)

[0, 0, 0, 0, 1, 0, 1, 1, 0, 0]


### 读取数据

In [168]:
X_raw=np.load('data/all.npy')

In [169]:
X_raw.shape

(8784, 18, 103)

In [170]:
X_raw[0].shape

(18, 103)

### X train

In [171]:
j=5
train_data=X_raw[:,j,:]

In [172]:
columns_with_missing_data = np.any(np.isnan(train_data), axis=0)
missing_columns = np.where(columns_with_missing_data)[0]
result = np.delete(train_data, missing_columns, axis=1)
# result = (result - np.min(result)) / (np.max(result) - np.min(result))

In [173]:
missing_columns

array([ 0, 34, 38, 49, 50, 55, 56, 73, 81, 83], dtype=int64)

In [174]:
result.shape

(8784, 93)

In [175]:
mean = result.mean(axis=0)
std = result.std(axis=0)
result = (result - mean) / (std + 1e-6)

In [176]:

train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71]
# train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]
# test_list=[52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76]
test_list=[72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92]

In [177]:
train_data=result[:,train_list]
test_data=result[:,test_list]

In [178]:
train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)

In [179]:
train_data.shape

(8784, 72, 1)

In [180]:
test_data.shape

(8784, 21, 1)

In [12]:
# train_data = np.transpose(train_data, (0, 2, 1))
# test_data = np.transpose(test_data, (0, 2, 1))

In [24]:
# train_data = np.reshape(train_data, (8784, 41,1))
# test_data = np.reshape(test_data, (8784, 52,1))

## Timestap

In [181]:
timestap_list=[0,1,2,3]

In [182]:
timestap=X_raw[:,timestap_list,:]

In [183]:
timestap=timestap.astype(np.int32)
timestap=timestap.astype(np.str_)

In [184]:
for i in range(len(timestap)):
    for m in range(1,4):
        timestap[i,m,1]=timestap[i,m,1].zfill(2)

In [185]:
timestap=timestap[:,:,1]
timestap_result=[]
for i in range(len(timestap)):
    x=timestap[i,0]+timestap[i,1]+timestap[i,2]+'-'+timestap[i,3]
    timestap_result.append(x)

In [186]:
timestap_result=np.array(timestap_result)

In [187]:
timestap_result.shape

(8784,)

### invid_mask掩码

In [188]:
def generate1_random_array(length):
    array = []
    for _ in range(length):
        array.append(False)
    return np.array(array)
 
# 生成长度为10的随机数组
train_invid_mask=[]
test_invid_mask=[]
for i in range(len(timestap)):
    random_array = generate1_random_array(train_data.shape[1])
    random_array1 = generate1_random_array(test_data.shape[1])
    train_invid_mask.append(random_array)
    test_invid_mask.append(random_array1)
train_invid_mask=np.array(train_invid_mask)
test_invid_mask=np.array(test_invid_mask)

### test_mask掩码

In [189]:
import random
 
def generate_random_array(length):
    array = []
    for _ in range(length):
            array.append(random.choice([False,True]))
    return np.array(array)

# 生成长度为10的随机数组
test_mask=[]
random_array=generate_random_array(test_data.shape[1])
for i in range(len(timestap)):
    test_mask.append(random_array)
test_mask=np.array(test_mask)

### 地理位置

#### 创建r_pos_mat

In [196]:
stations = pd.read_csv("D:\Spatial_interpolation\SSIN\data\Station_info.csv")

In [197]:
stations.drop([0, 34, 38, 49, 50, 55, 56, 73, 81, 83],inplace=True)

In [198]:
df_train=stations.iloc[0:72]
df_test=stations.iloc[72:93]

In [199]:
def calc_dist_angle_mat(df, out_path):
    lons, lats = df["lon"].values, df["lat"].values
    dist_angle_mat = np.zeros((len(lons), len(lons), 2))

    for i in range(len(lons)):
        for j in range(len(lons)):
            dist = Geodesic.WGS84.Inverse(lats[i], lons[i], lats[j], lons[j])
            dist_angle_mat[i, j, 0] = dist["s12"] / 1000.0  # distance, km
            dist_angle_mat[i, j, 1] = dist["azi1"]  # azimuth at the first point in degrees

    print(dist_angle_mat.shape)
    # print(dist_angle_mat)
    np.save(out_path, dist_angle_mat)

In [200]:
if __name__ == "__main__":
    base_dir = "../data"

    # HK dataset
    # info_path = f"{base_dir}/HK_123_data/hko_stations_info.csv"
    # out_dir = f"{base_dir}"

    # BW dataset
    # info_path = f"{base_dir}/BW_132_data/BW_stations_info.csv"
    out_dir = f"{base_dir}"

    # os.makedirs(out_dir, exist_ok=True)

    out_name = "dist_angle_mat_train.npy"
    out_path = f"{out_dir}/{out_name}"
    calc_dist_angle_mat(df_train, out_path)

(72, 72, 2)


In [201]:
if __name__ == "__main__":
    base_dir = "../data"

    # HK dataset
    # info_path = f"{base_dir}/HK_123_data/hko_stations_info.csv"
    out_dir = f"{base_dir}"

    # BW dataset
    # info_path = f"{base_dir}/BW_132_data/BW_stations_info.csv"
    # out_dir = f"{base_dir}/BW_132_data"

    # os.makedirs(out_dir, exist_ok=True)

    out_name = "dist_angle_mat_test.npy"
    out_path = f"{out_dir}/{out_name}"
    calc_dist_angle_mat(df_test, out_path)

(21, 21, 2)


In [202]:
def generate_stat_from_train_data(train_info_df, r_pos_mat):
    # info_df = pd.read_csv(info_path)
    # ori_r_pos_mat = np.load(relative_pos_mat_path)

    # is_test = info_df["is_test"].values
    # train_mask = np.where(is_test == 0, True, False)
    # train_info_df = info_df.loc[train_mask, :]

    lat_mean, lat_std, lat_max, lat_min = train_info_df["lat"].mean(), train_info_df["lat"].std(ddof=0), \
                                          train_info_df["lat"].max(), train_info_df["lat"].min()
    lon_mean, lon_std, lon_max, lon_min = train_info_df["lon"].mean(), train_info_df["lon"].std(ddof=0), \
                                          train_info_df["lon"].max(), train_info_df["lon"].min()

    # indexes = np.where(train_mask)[0]
    # idx_i, idx_j = np.ix_(indexes, indexes)
    # r_pos_mat = ori_r_pos_mat[idx_i, idx_j, :]

    r_dist_mat = r_pos_mat[:, :, 0]
    r_angle_mat = r_pos_mat[:, :, 1]

    r_dist_mean, r_dist_std, r_dist_max, r_dist_min = np.mean(r_dist_mat), np.std(r_dist_mat), \
                                                      np.max(r_dist_mat), np.min(r_dist_mat),
    r_angle_mean, r_angle_std, r_angle_max, r_angle_min = np.mean(r_angle_mat), np.std(r_angle_mat), \
                                                          np.max(r_angle_mat), np.min(r_angle_mat)

    stat_dict = {}
    stat_dict["lat_mean"], stat_dict["lat_std"], stat_dict["lat_max"], stat_dict["lat_min"] = \
        lat_mean, lat_std, lat_max, lat_min
    stat_dict["lon_mean"], stat_dict["lon_std"], stat_dict["lon_max"], stat_dict["lon_min"] = \
        lon_mean, lon_std, lon_max, lon_min

    stat_dict["r_dist_mean"], stat_dict["r_dist_std"], stat_dict["r_dist_max"], stat_dict["r_dist_min"] = \
        r_dist_mean, r_dist_std, r_dist_max, r_dist_min
    stat_dict["r_angle_mean"], stat_dict["r_angle_std"], stat_dict["r_angle_max"], stat_dict["r_angle_min"] = \
        r_angle_mean, r_angle_std, r_angle_max, r_angle_min

    print("Calculates the statistics of training data. Done!")

    # with open("./data/hk_data_stats.pkl".format(out_name), "wb") as fp:
    #     pickle.dump(stat_dict, fp)

    return stat_dict

In [203]:
train_df=np.load("D:\Spatial_interpolation\data\dist_angle_mat_train.npy")
test_df=np.load("D:\Spatial_interpolation\data\dist_angle_mat_test.npy")

In [204]:
stat_dict=generate_stat_from_train_data(df_train, train_df)

Calculates the statistics of training data. Done!


In [205]:

train_df[:, :, 0] = (train_df[:, :, 0] - stat_dict["r_dist_mean"]) / stat_dict["r_dist_std"]
train_df[:, :, 1] = (train_df[:, :, 1] - stat_dict["r_angle_mean"]) / stat_dict["r_angle_std"]

In [206]:
test_df[:, :, 0] = (test_df[:, :, 0] - stat_dict["r_dist_mean"]) / stat_dict["r_dist_std"]
test_df[:, :, 1] = (test_df[:, :, 1] - stat_dict["r_angle_mean"]) / stat_dict["r_angle_std"]

### 创建最后的结果

In [207]:
test={"test_data":test_data,"invalid_masks":test_invid_mask,"test_masks":test_mask,"r_pos_mat":test_df,"timestamps":timestap_result}

In [208]:
train={"train_data":train_data,"invalid_masks":train_invid_mask,"r_pos_mat":train_df,"timestamps":timestap_result}

In [209]:
print(list(test.keys()))

['test_data', 'invalid_masks', 'test_masks', 'r_pos_mat', 'timestamps']


In [210]:
print(list(train.keys()))

['train_data', 'invalid_masks', 'r_pos_mat', 'timestamps']


In [211]:
with open('train.pkl', 'wb') as f:
    pickle.dump(train, f)


with open('test.pkl', 'wb') as f:
    pickle.dump(test, f)

### test

In [34]:
# import pandas as pd

# data = pd.read_pickle("2012-2014_data_train.pkl")


In [6]:
# data_bw_train = pd.read_pickle("2012-2014_data_bw_train.pkl")
# data_bw_test = pd.read_pickle("2012-2014_data_bw_test.pkl")

In [17]:
# data.keys()

dict_keys(['test_data', 'invalid_masks', 'test_masks', 'r_pos_mat', 'timestamps'])

In [18]:
# data_bw_train.keys()

dict_keys(['test_data', 'invalid_masks', 'test_masks', 'r_pos_mat', 'timestamps'])

In [26]:
# data_bw_train["r_pos_mat"].shape

(132, 132, 2)

In [36]:
# data["invalid_masks"].shape

(8784, 41)